# Running BAGPIPES on a DJA NIRSpec/PRISM spectrum

Browse DJA v3 spectra here: https://s3.amazonaws.com/msaexp-nirspec/extractions/nirspec_graded_v3.html

In [1]:
# ------- LIBRARIES ------ #
import db_pull_funcs as db_pull
import pipes_fitting_funcs as p_fit
import pipes_plotting_funcs as p_plot

import bagpipes as pipes
import numpy as np

from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck13 as cosmo

import os

Bagpipes: Latex distribution not found, plots may look strange.


In [2]:
if not os.path.exists("./files"):
    os.mkdir("./files")

### fitting params

In [3]:
def fitting_params(z_spec, sfh="continuity", scale_disp=1.3, use_msa_resamp=False, fit_agn=False, fit_dla=False):

    fit_instructions = {}
    
    ## ---------- ## double power-law sfh (parametric)
    dblplaw = {}                        
    dblplaw["tau"] = (0., 15.)            
    dblplaw["alpha"] = (0.01, 1000.)
    dblplaw["beta"] = (0.01, 1000.)
    dblplaw["alpha_prior"] = "log_10"
    dblplaw["beta_prior"] = "log_10"
    dblplaw["massformed"] = (1., 13.)
    dblplaw["metallicity"] = (0.003, 2.)
    dblplaw["metallicity_prior"] = "log_10"

    
    ## ---------- ## delayed-tau sfh (parametric)
    delayed = {}                         # Delayed Tau model t*e^-(t/tau)
    delayed["age"] = (0.1, 9.)           # Time since SF began: Gyr
    delayed["tau"] = (0.1, 9.)           # Timescale of decrease: Gyr
    delayed["massformed"] = (6., 13.)
    delayed["metallicity"] = (0.2, 1.)   # in Zsun

    
    ## ---------- ## continuity sfh (non-parametric)
    def get_age_bins(z):
        max_age = cosmo.age(z).to('Myr').value - cosmo.age(20).to('Myr').value
        age_bins = [0., 10.,]
        for i in np.logspace(1.5, np.log10(max_age), 6):
            age_bins.append(i)
        
        return age_bins
    
    continuity = {}
    continuity["massformed"] = (6., 14.)
    continuity["metallicity"] = (0.01, 3.)
    continuity["metallicity_prior"] = "log_10"
    continuity["bin_edges"] = get_age_bins(z_spec)
    
    for i in range(1, len(continuity["bin_edges"])-1):
        continuity["dsfr" + str(i)] = (-10., 10.)
        continuity["dsfr" + str(i) + "_prior"] = "student_t"
        continuity["dsfr" + str(i) + "_prior_scale"] = 1.0  # Defaults to 0.3 as in Leja19, but can be set - 1 is bursty continuity prior from Tacchella+21
        continuity["dsfr" + str(i) + "_prior_df"] = 2       # Defaults to this value as in Leja19, but can be set
    
    continuity["age_min"] = 0


    # setting the preferred sfh
    if sfh=="continuity":
        fit_instructions["continuity"] = continuity
    elif sfh=="dblplaw":
        fit_instructions["dblplaw"] = dblplaw
    elif sfh=="delayed":
        fit_instructions["delayed"] = delayed

    
    ## ---------- ## nebular emisison, logU
    nebular = {}
    nebular["logU"] = (-4., -1.)
    nebular["logU_prior"] = "Gaussian"
    nebular["logU_prior_mu"] = -2.25
    nebular["logU_prior_sigma"] = 0.25
    fit_instructions["nebular"] = nebular

    
    ## ---------- ## dust law
    dust = {}
    dust["type"] = "Calzetti"            # Define the shape of the attenuation curve
    dust["Av"] = (0., 4.)                # Vary Av between 0 and 4 magnitudes
    # dust["type"] = "CF00"
    # dust["eta"] = 2.
    # dust["Av"] = (0., 2.0)
    # dust["n"] = (0.3, 2.5)
    # dust["n_prior"] = "Gaussian"
    # dust["n_prior_mu"] = 0.7
    # dust["n_prior_sigma"] = 0.3
    fit_instructions["dust"] = dust

    
    ## ---------- ## agn component
    agn = {}
    agn["alphalam"] = (-2., 2.)
    agn["betalam"] = (-2., 2.)
    agn["f5100A"] = (0, 1e-19)
    agn["sigma"] = (1e3, 5e3)
    agn["hanorm"] = (0,2.5e-17)

    if fit_agn:
        fit_instructions["agn"] = agn
    
    
    ## ---------- ## tight redshift prior around z_spec
    fit_instructions["redshift"] = (z_spec - 0.001*(1+z_spec), z_spec + 0.001*(1+z_spec))
    fit_instructions["redshift_prior"] = "Gaussian"
    fit_instructions["redshift_prior_mu"] = z_spec
    fit_instructions["redshift_prior_sigma"] = 0.001 * (1+z_spec)

    
    ## ---------- ## jwst prism resolution curve
    hdul = fits.open("jwst_nirspec_prism_disp.fits")
    resData = np.c_[10000*hdul[1].data["WAVELENGTH"], hdul[1].data["R"]*scale_disp]
    fit_instructions["R_curve"] = resData

    
    ## ---------- ## boolean for using msa resampling
    fit_instructions["use_msa_resamp"] = use_msa_resamp

    
    ## ---------- ## fixed velocity dispersion
    fit_instructions["veldisp"] = 100.   #km/s

    ## ---------- ## dla component (doesn't work?)
    dla = {}
    dla["zabs"] = z_spec
    dla["t"] = 22.
    if fit_dla:
        fit_instructions["dla"] = dla

    
    ## ---------- ## calibration curve (2nd order polynomial)
    calib = {}
    calib["type"] = "polynomial_bayesian"
    
    calib["0"] = (0.5, 1.5) # Zero order is centred on 1, at which point there is no change to the spectrum.
    calib["0_prior"] = "Gaussian"
    calib["0_prior_mu"] = 1.0
    calib["0_prior_sigma"] = 0.25
    
    calib["1"] = (-0.5, 0.5) # Subsequent orders are centred on zero.
    calib["1_prior"] = "Gaussian"
    calib["1_prior_mu"] = 0.
    calib["1_prior_sigma"] = 0.25
    
    calib["2"] = (-0.5, 0.5)
    calib["2_prior"] = "Gaussian"
    calib["2_prior_mu"] = 0.
    calib["2_prior_sigma"] = 0.25
    
    fit_instructions["calib"] = calib

    
    ## ---------- ##
    mlpoly = {}
    mlpoly["type"] = "polynomial_max_like"
    mlpoly["order"] = 2


    ## ---------- ## white noise scaling
    noise = {}
    noise["type"] = "white_scaled"
    noise["scaling"] = (1., 10.)
    noise["scaling_prior"] = "log_10"
    fit_instructions["noise"] = noise

   
    

    return(fit_instructions)

In [4]:
# speclist_cat = Table.read('spec_cat_temp.fits', format='fits')
# id=1
# speclist_cat[speclist_cat["id"]==id]["fname"][0]+'.spec.fits'
# # spec_cat

In [5]:
fnames = ['rubies-egs61-v3_prism-clear_4233_75646',
          'rubies-uds41-v3_prism-clear_4233_43941',
          'rubies-uds31-v3_prism-clear_4233_166283',
          'gds-barrufet-s67-v3_prism-clear_2198_6620',
          'ceers-ddt-v3_prism-clear_2750_28',
          'rubies-egs51-v3_prism-clear_4233_19489',
          'gds-barrufet-s67-v3_prism-clear_2198_5446',
          'rubies-uds3-v3_prism-clear_4233_62812',
          'rubies-egs63-v3_prism-clear_4233_49140']
ids = [0,1,2,3,4,5,6,7,8]

# catalogue of spectra from DJA
speclist_cat = Table([ids, fnames], names=list(['id', 'fname']))
speclist_cat.write('spec_cat_temp.fits', format='fits', overwrite=True)

In [6]:
def run_pipes_on_dja_spec(spec_cat):
    """
    Runs bagpipes on spectrum from DJA AWS database, saves posteriors as .h5 files and plots as .pdf files
    
    Parameters
    ----------
    spec_cat : catalogue of spectra and corresponding ids, format=astropy Table

    Returns
    -------
    None
    """

    # name of bagpipes run
    runName = 'pipes_test'

    # make pipes folders if they don't already exist
    pipes.utils.make_dirs(run=runName)

    ##################################
    # ---- PULLING DATA FROM DB ---- #
    ##################################

    # id and dja name of spectrum
    runid, fname = spec_cat

    print(str(runid))

    # spectrum and photometry filenames
    fname_spec = fname+'.spec.fits'
    fname_phot = fname+'.phot.cat'

    # path to store spectrum and photometry files
    filePath = 'files/'

    # pull spectrum and photometry from the aws database
    db_pull.pull_spec_from_db(fname_spec, filePath)
    db_pull.pull_phot_from_db(fname_spec, fname_phot, filePath)

    # spectroscopic redshift
    z_spec = db_pull.pull_zspec_from_db(fname_spec)

    # save data to plot
    p_plot.plot_spec_phot_data(fname_spec, fname_phot, f_lam=True, show=False, save=True, run=runName)

    ##################################
    # -------- BAGPIPES RUN -------- #
    ##################################

    # jwst filter list
    filt_list = np.loadtxt("../filters/filt_list.txt", dtype="str")

    # making galaxy object
    galaxy = pipes.galaxy(runid, p_fit.load_both, filt_list=filt_list,
                        spec_units='ergscma',
                        phot_units='muJy',
                        out_units="ergscma")

    # generating fit instructions
    fit_instructions = fitting_params(z_spec, sfh="continuity")

    # making fit object
    fit = pipes.fit(galaxy, fit_instructions, run=runName)

    # fitting  spectrum and photometry with bagpipes
    fit.fit(verbose=False, sampler='nautilus', pool=10)

    ##################################
    # ---------- PLOTTING ---------- #
    ##################################

    # fitted model
    fig = p_plot.plot_fitted_spectrum(fit, fname_spec, f_lam=True, save=True)
    # star-formation history
    fig = p_plot.plot_fitted_sfh(fit, fname_spec, save=True)
    # posterior corner plot
    fig = fit.plot_corner(save=True, show=False)

    # saving posterior quantities to table
    tab = p_plot.get_posterior_sample_dists(fit, fname_spec, save=True)

    return fit

In [7]:
fit = run_pipes_on_dja_spec(speclist_cat[3])

3

Bagpipes: fitting object 3

Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: Latex distribution not found, plots may look strange.

Completed in 167.2 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
calib:0                        1.350      1.379      1.409
calib:1                       -0.001      0.032      0.068
calib: